# Basic usage of LRP

We start out by loading a small convolutional neural network:

In [1]:
using RelevancePropagation
using Flux

model = Chain(
    Chain(
        Conv((3, 3), 3 => 8, relu; pad=1),
        Conv((3, 3), 8 => 8, relu; pad=1),
        MaxPool((2, 2)),
        Conv((3, 3), 8 => 16; pad=1),
        BatchNorm(16, relu),
        Conv((3, 3), 16 => 8, relu; pad=1),
        BatchNorm(8, relu),
    ),
    Chain(Flux.flatten, Dense(2048 => 512, relu), Dropout(0.5), Dense(512 => 100, softmax)),
);

This model contains two chains: the convolutional layers and the fully connected layers.

## Model preparation

### Stripping the output softmax
When using LRP, it is recommended to explain output logits instead of probabilities.
This can be done by stripping the output softmax activation from the model
using the `strip_softmax` function:

In [2]:
model = strip_softmax(model)

Chain(
  Chain(
    Conv((3, 3), 3 => 8, relu, pad=1),  # 224 parameters
    Conv((3, 3), 8 => 8, relu, pad=1),  # 584 parameters
    MaxPool((2, 2)),
    Conv((3, 3), 8 => 16, pad=1),       # 1_168 parameters
    BatchNorm(16, relu),                # 32 parameters, plus 32
    Conv((3, 3), 16 => 8, relu, pad=1),  # 1_160 parameters
    BatchNorm(8, relu),                 # 16 parameters, plus 16
  ),
  Chain(
    Flux.flatten,
    Dense(2048 => 512, relu),           # 1_049_088 parameters
    Dropout(0.5),
    Dense(512 => 100),                  # 51_300 parameters
  ),
)         # Total: 16 trainable arrays, 1_103_572 parameters,
          # plus 4 non-trainable, 48 parameters, summarysize 4.213 MiB.

If you don't remove the output softmax,
model checks will fail.

### Canonizing the model
LRP is not invariant to a model's implementation.
Applying the `GammaRule` to two linear layers in a row will yield different results
than first fusing the two layers into one linear layer and then applying the rule.
This fusing is called "canonization" and can be done using the `canonize` function:

In [3]:
model_canonized = canonize(model)

Chain(
  Conv((3, 3), 3 => 8, relu, pad=1),    # 224 parameters
  Conv((3, 3), 8 => 8, relu, pad=1),    # 584 parameters
  MaxPool((2, 2)),
  Conv((3, 3), 8 => 16, relu, pad=1),   # 1_168 parameters
  Conv((3, 3), 16 => 8, relu, pad=1),   # 1_160 parameters
  BatchNorm(8, relu),                   # 16 parameters, plus 16
  Flux.flatten,
  Dense(2048 => 512, relu),             # 1_049_088 parameters
  Dropout(0.5),
  Dense(512 => 100),                    # 51_300 parameters
)         # Total: 14 trainable arrays, 1_103_540 parameters,
          # plus 2 non-trainable, 16 parameters, summarysize 4.212 MiB.

After canonization, the first `BatchNorm` layer has been fused into the preceding `Conv` layer.
The second `BatchNorm` layer wasn't fused
since its preceding `Conv` layer has a ReLU activation function.

### Flattening the model
RelevancePropagation.jl's LRP implementation supports nested Flux Chains and Parallel layers.
However, it is recommended to flatten the model before analyzing it.

LRP is implemented by first running a forward pass through the model,
keeping track of the intermediate activations, followed by a backward pass
that computes the relevances.

To keep the LRP implementation simple and maintainable,
RelevancePropagation.jl does not pre-compute "nested" activations.
Instead, for every internal chain, a new forward pass is run to compute activations.

By "flattening" a model, this overhead can be avoided.
For this purpose, RelevancePropagation.jl provides the function `flatten_model`:

In [4]:
model_flat = flatten_model(model)

Chain(
  Conv((3, 3), 3 => 8, relu, pad=1),    # 224 parameters
  Conv((3, 3), 8 => 8, relu, pad=1),    # 584 parameters
  MaxPool((2, 2)),
  Conv((3, 3), 8 => 16, pad=1),         # 1_168 parameters
  BatchNorm(16, relu),                  # 32 parameters, plus 32
  Conv((3, 3), 16 => 8, relu, pad=1),   # 1_160 parameters
  BatchNorm(8, relu),                   # 16 parameters, plus 16
  Flux.flatten,
  Dense(2048 => 512, relu),             # 1_049_088 parameters
  Dropout(0.5),
  Dense(512 => 100),                    # 51_300 parameters
)         # Total: 16 trainable arrays, 1_103_572 parameters,
          # plus 4 non-trainable, 48 parameters, summarysize 4.212 MiB.

This function is called by default when creating an LRP analyzer.
Note that we pass the unflattened model to the analyzer, but `analyzer.model` is flattened:

In [5]:
analyzer = LRP(model)
analyzer.model

Chain(
  Conv((3, 3), 3 => 8, relu, pad=1),    # 224 parameters
  Conv((3, 3), 8 => 8, relu, pad=1),    # 584 parameters
  MaxPool((2, 2)),
  Conv((3, 3), 8 => 16, pad=1),         # 1_168 parameters
  BatchNorm(16, relu),                  # 32 parameters, plus 32
  Conv((3, 3), 16 => 8, relu, pad=1),   # 1_160 parameters
  BatchNorm(8, relu),                   # 16 parameters, plus 16
  Flux.flatten,
  Dense(2048 => 512, relu),             # 1_049_088 parameters
  Dropout(0.5),
  Dense(512 => 100),                    # 51_300 parameters
)         # Total: 16 trainable arrays, 1_103_572 parameters,
          # plus 4 non-trainable, 48 parameters, summarysize 4.212 MiB.

If this flattening is not desired, it can be disabled
by passing the keyword argument `flatten=false` to the `LRP` constructor.

## LRP rules
By default, the `LRP` constructor will assign the `ZeroRule` to all layers.

In [6]:
LRP(model)

LRP(
  Conv((3, 3), 3 => 8, relu, pad=1)  => ZeroRule(),
  Conv((3, 3), 8 => 8, relu, pad=1)  => ZeroRule(),
  MaxPool((2, 2))                    => ZeroRule(),
  Conv((3, 3), 8 => 16, pad=1)       => ZeroRule(),
  BatchNorm(16, relu)                => ZeroRule(),
  Conv((3, 3), 16 => 8, relu, pad=1) => ZeroRule(),
  BatchNorm(8, relu)                 => ZeroRule(),
  Flux.flatten                       => ZeroRule(),
  Dense(2048 => 512, relu)           => ZeroRule(),
  Dropout(0.5)                       => ZeroRule(),
  Dense(512 => 100)                  => ZeroRule(),
)

This analyzer will return heatmaps that look identical to the `InputTimesGradient` analyzer
from [ExplainableAI.jl](https://github.com/Julia-XAI/ExplainableAI.jl).

LRP's strength lies in assigning different rules to different layers,
based on their functionality in the neural network[^1].
RelevancePropagation.jl implements many LRP rules out of the box,
but it is also possible to *implement custom rules*.

To assign different rules to different layers,
use one of the composites presets,
or create your own composite, as described in
*Assigning rules to layers*.

In [7]:
composite = EpsilonPlusFlat() # using composite preset EpsilonPlusFlat

Composite(
  GlobalTypeMap(  # all layers
    Flux.Conv               => ZPlusRule(),
    Flux.ConvTranspose      => ZPlusRule(),
    Flux.CrossCor           => ZPlusRule(),
    Flux.Dense              => EpsilonRule{Float32}(1.0f-6),
    typeof(NNlib.dropout)   => PassRule(),
    Flux.AlphaDropout       => PassRule(),
    Flux.Dropout            => PassRule(),
    Flux.BatchNorm          => PassRule(),
    typeof(Flux.flatten)    => PassRule(),
    typeof(MLUtils.flatten) => PassRule(),
    typeof(identity)        => PassRule(),
 ),
  FirstLayerTypeMap(  # first layer
    Flux.Conv          => FlatRule(),
    Flux.ConvTranspose => FlatRule(),
    Flux.CrossCor      => FlatRule(),
 ),
)

In [8]:
LRP(model, composite)

LRP(
  Conv((3, 3), 3 => 8, relu, pad=1)  => FlatRule(),
  Conv((3, 3), 8 => 8, relu, pad=1)  => ZPlusRule(),
  MaxPool((2, 2))                    => ZeroRule(),
  Conv((3, 3), 8 => 16, pad=1)       => ZPlusRule(),
  BatchNorm(16, relu)                => PassRule(),
  Conv((3, 3), 16 => 8, relu, pad=1) => ZPlusRule(),
  BatchNorm(8, relu)                 => PassRule(),
  Flux.flatten                       => PassRule(),
  Dense(2048 => 512, relu)           => EpsilonRule{Float32}(1.0f-6),
  Dropout(0.5)                       => PassRule(),
  Dense(512 => 100)                  => EpsilonRule{Float32}(1.0f-6),
)

## Computing layerwise relevances
If you are interested in computing layerwise relevances,
call `analyze` with an LRP analyzer and the keyword argument
`layerwise_relevances=true`.

The layerwise relevances can be accessed in the `extras` field
of the returned `Explanation`:

In [9]:
input = rand(Float32, 32, 32, 3, 1) # dummy input for our convolutional neural network

expl = analyze(input, analyzer; layerwise_relevances=true)
expl.extras.layerwise_relevances

(Float32[-1.3243921 8.74806 … 2.8880134 -1.1207205; 0.45582312 1.2697315 … -2.5142088 0.29030505; … ; 1.6582028 10.617342 … -1.9481914 -0.29652497; 1.0133505 2.7148626 … -0.6336324 -0.7926021;;; 2.0289943 -0.83887166 … 0.17195909 0.5172456; 2.6238258 -4.3241253 … -1.8676149 5.0231013; … ; 1.403176 3.2535462 … -2.1448593 -0.2184069; 1.7888104 -1.822815 … -2.7161741 2.4246175;;; -1.3276355 -4.0249233 … -0.17526749 -0.6048343; -0.37539476 3.2393668 … -0.55159587 3.74323; … ; -7.081163 -1.1179627 … 1.6861738 0.07048841; -0.215447 0.21218763 … -3.0160213 5.199525;;;;], Float32[-2.128875 -4.413198 … 1.0796653 -1.3906882; -0.0 -0.25499406 … -0.0 0.0; … ; 0.0 -0.0 … 0.0 4.310081; 0.68768483 -1.3531804 … -0.08816633 -1.0259305;;; 1.4914618 -0.2784944 … -0.08433416 0.0; -2.834426 -0.7316214 … 0.0 0.0; … ; -2.492968 -3.0856326 … 6.881095 -0.0; 0.0 0.0 … 0.0 0.51989573;;; 0.0 0.12901542 … 0.0 2.8268218; -0.0 0.0 … 1.2645891 2.2773566; … ; -0.0 4.127004 … 0.9125523 0.13013807; 0.32082933 0.08839007

Note that the layerwise relevances are only kept for layers in the outermost `Chain` of the model.
When using our unflattened model, we only obtain three layerwise relevances,
one for each chain in the model and the output relevance:

In [10]:
analyzer = LRP(model; flatten=false) # use unflattened model

expl = analyze(input, analyzer; layerwise_relevances=true)
expl.extras.layerwise_relevances

(Float32[-0.06669502 -0.4572314 … -0.068132564 0.019794917; -0.02946696 -0.08770134 … 0.0119196465 0.047512513; … ; 1.4059473 3.2203827 … -16.343994 2.72469; -0.2982892 -0.34363398 … -19.85114 18.02567;;; -0.7724999 -0.44059408 … 0.018334314 0.030536616; -0.17616431 0.35781795 … -0.034816585 -0.068525895; … ; 0.29185158 -5.1027412 … 4.185837 -1.0520825; 0.8141925 -1.1828058 … -0.0393799 10.90266;;; 0.02395865 1.1562291 … 0.0010136584 -0.06801985; -0.5289601 -0.25900972 … 0.124358915 -0.004795539; … ; 0.08604528 -0.9689742 … 16.290344 -6.422851; 0.012542506 -0.059912365 … -14.378358 -19.239777;;;;], Float32[-0.0013751079 0.001177785 … 0.00046929833 0.0; -0.004867432 0.00030990306 … -0.0 0.0; … ; -0.004230966 -0.0007762051 … 0.0 0.0; -0.00487678 -0.015785785 … 0.0012445684 0.0;;; 0.0013436094 -0.0 … -0.0 0.0; -0.011231522 0.0 … -0.0 -0.0; … ; -0.0 -0.0 … -0.0 -0.0016579343; 0.0 0.0 … -0.0 0.0046614176;;; -0.0005816056 -0.0 … -0.0 -0.0; 0.00453179 -0.0 … 0.0010710389 -0.0; … ; -0.00349304

## Performance tips
### Using LRP with a GPU
All LRP analyzers support GPU backends,
building on top of [Flux.jl's GPU support](https://fluxml.ai/Flux.jl/stable/gpu/).
Using a GPU only requires moving the input array and model weights to the GPU.

For example, using [CUDA.jl](https://github.com/JuliaGPU/CUDA.jl):

```julia
using CUDA, cuDNN
using Flux
using ExplainableAI

# move input array and model weights to GPU
input = input |> gpu # or gpu(input)
model = model |> gpu # or gpu(model)

# analyzers don't require calling `gpu`
analyzer = LRP(model)

# explanations are computed on the GPU
expl = analyze(input, analyzer)
```

Some operations, like saving, require moving explanations back to the CPU.
This can be done using Flux's `cpu` function:

```julia
val = expl.val |> cpu # or cpu(expl.val)

using BSON
BSON.@save "explanation.bson" val
```

### Using LRP without a GPU
Using Julia's package extension mechanism,
RelevancePropagation.jl's LRP implementation can optionally make use of
[Tullio.jl](https://github.com/mcabbott/Tullio.jl) and
[LoopVectorization.jl](https://github.com/JuliaSIMD/LoopVectorization.jl)
for faster LRP rules on dense layers.

This only requires loading the packages before loading RelevancePropagation.jl:
```julia
using LoopVectorization, Tullio
using RelevancePropagation
```

[^1]: G. Montavon et al., [Layer-Wise Relevance Propagation: An Overview](https://link.springer.com/chapter/10.1007/978-3-030-28954-6_10)

---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*